In [ ]:
import requests

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
    'Accept': '*/*',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Dest': 'script',
    'Referer': 'https://authserver.whsw.cn/',
    'Accept-Language': 'zh-CN,zh;q=0.9',
}

params = {
    'callback': 'cx_captcha_function',
    'captchaId': 'qDG21VMg9qS5Rcok4cfpnHGnpf5LhcAv',
    'type': 'slide',
    'version': '1.1.20',
    'captchaKey': '25585b92407433c32595c189bbf261b4',
    'token': '90d750d57d8290bae870813af32afc2e:1741453544490',
    'referer': 'https://authserver.whsw.cn/cas/login?service=https%3A%2F%2Fauthserver.whsw.cn%2Flogin',
    'iv': '9a379f1f3a67256d42e4cd0c7a1aa4c1',
    '_': '1741445019219',
}

response = requests.get('https://captcha.chaoxing.com/captcha/get/verification/image', params=params, headers=headers)

In [ ]:
import requests
import pandas as pd
import re
import ddddocr
import uuid
import time
import hashlib

#### 类
#### 登录
#### 验证码处理：
1. 获取conf接口，获取服务器时间戳返回
2. 获取验证码
* 解决4个变化的参数
* 获取背景图，方块图
* 参数的加密处理
3. 验证验证码
* 背景图和缺口的距离 X （识别）——参数加密处理
* 获取一个验证成功的参数
4. 登录

In [ ]:
class SlidePass(object):
    def __init__(self):
        self.headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
    'Accept': '*/*',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Dest': 'script',
    'Referer': 'https://authserver.whsw.cn/',
    'Accept-Language': 'zh-CN,zh;q=0.9',}

    # 请求工具方法    
    def Toolrequests(self,url,method=None,headers="",params=None,data=None):
        
            #请求工具方法
            if method == "get":
                response= requests.get(url,headers=headers,params=params)
                return response
            elif method == "post":
                response = requests.post(url,headers=headers,data=data)
                return response
            else:
                print("请求方法错误")

    def md5_encrypt(self,text):
        # 创建md5对象
        md5 = hashlib.md5()
        
        # 对字符串进行编码，然后更新md5对象
        md5.update(text.encode('utf-8'))
        
        # 获取加密后的16进制字符串
        return md5.hexdigest()

    # 获取服务器时间戳
    def get_severtime(self):
        url = "https://captcha.chaoxing.com/captcha/get/conf"
        params = {   
            'callback': 'cx_captcha_function',
            'captchaId': 'qDG21VMg9qS5Rcok4cfpnHGnpf5LhcAv',
            '_': str(round(time.time()*1000,))         #本地时间戳
            }
        response = self.Toolrequests(url,method="get",headers=self.headers,params=params).text
        severtime = response[25:38]
        return severtime                                                          
    # 获取验证码的背景、方块
    def get_slide(self):
        servertime = self.get_severtime()
        uuid4 =  uuid.uuid4()
        url = "https://captcha.chaoxing.com/captcha/get/verification/image"
        captchaId = "qDG21VMg9qS5Rcok4cfpnHGnpf5LhcAv"
        captchaKey = self.md5_encrypt(servertime+str(uuid4))
        token = self.md5_encrypt(servertime+captchaId+'slide'+captchaKey)+ ':' + (int(servertime) + 0x493e0)
        iv = self.md5_encrypt(captchaId+'slide'+str(round(time.time()*1000,))+str(uuid4))

        params = {
            'callback': 'cx_captcha_function',
            'captchaId': 'qDG21VMg9q5SRcok4cfpnHGnpf5LhcAv',
            'type': 'slide',
            'version': '1.1.20',
            'captchaKey': captchaKey,
            'token': token,
            'referer': 'https://authserver.whsw.cn/cas/login?service=https://whsw.jw.chaoxing.com/admin/caslogin',
            'iv': iv,
            '_': str(round(time.time()*1000,))
            }

        response = self.Toolrequests(url,method="get",headers=self.headers,params=params).text
        # 请求成功的token'
        token_result = re.findall(r'"token":"(.*?)"',response)[0]
        # 背景图
        shadeImage = re.findall(r'"shadeImage":"(.*?)"',response)[0]
        # 方块图
        cutoutImage = re.findall(r'"cutoutImage":"(.*?)"',response)[0]

    def get_slide_pass(self):
        pass



    def get_encryption(self):
        pass
